In [1]:
from collections import defaultdict
import numpy as np
import random
import asyncio
import itertools
import matplotlib.pyplot as plt
from math import sqrt
from scipy.spatial.distance import pdist, squareform, euclidean
from scipy.special import gamma
from itertools import combinations, product
from sklearn.cluster import DBSCAN
import pandas as pd
from sklearn.metrics import f1_score, mean_squared_error

In [2]:
data_0 = np.loadtxt("1.txt").tolist()

In [3]:
def clusters_create(data_0):
    data = []
    for i in data_0:
        data.append([i, 0])
    clustering = DBSCAN(eps=0.01, min_samples=2).fit(data)
    labels = clustering.labels_
    labels = labels.tolist()
    
    lens = max(labels)
    clusters = []
    for i in range(lens + 2):
        clusters.append([])
        
    for i in range(len(data)):
        clusters[labels[i] + 1].append(data[i])
    
    maxim = 0
    max_len = 0
    for i in range(len(clusters)):
        if len(clusters[i]) > max_len:
            maxim = i
            max_len = len(clusters[i])
    
    return np.average(clusters[maxim])

In [4]:
clusters_create(data_0)

0.22442149328014874

In [5]:
def remove_points(my_list, percentage):
    new_list = [0] * len(my_list)
    todrop = random.sample(range(len(my_list)), len(my_list) - round(len(my_list)*percentage))
    for i in range(len(my_list)):
        if i not in todrop:
            new_list[i] = False
        else:
            new_list[i] = my_list[i]
    return new_list

In [6]:
new_list = remove_points(data_0, 0.5)
print(len(new_list), new_list.count(False))
new_list

100 50


[False,
 False,
 False,
 False,
 0.8071305178817035,
 0.6126963873590359,
 0.5254099874487781,
 False,
 False,
 0.5328470404642187,
 0.5779595875223092,
 False,
 False,
 False,
 0.7251716677626908,
 0.5417571554051449,
 False,
 0.41928271334425077,
 0.344071120341981,
 False,
 0.0965291085623442,
 0.22458977569720417,
 0.3800476974394521,
 0.4343166076914131,
 0.4169444980615153,
 0.3469985963802576,
 False,
 0.09278691560790445,
 0.21773255747180909,
 0.38166614469457594,
 0.44656714398059527,
 0.43841238241324537,
 False,
 False,
 False,
 0.1400710824199098,
 0.3259959410028469,
 0.4384356908046919,
 False,
 False,
 0.4125869390371602,
 0.32687804505518603,
 0.16279961945534052,
 0.09461875121038148,
 0.2634971514871307,
 False,
 0.4882031884806504,
 False,
 False,
 0.6327501549479992,
 False,
 0.9095651026297038,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 0.3917698629045777,
 False,
 0.5029456394936507,
 False,
 False,
 False,
 0.8290831107240422,
 Fals